## Write a Web Service

Wrap the output of the second exercise in a web service that returns the data in JSON format (instead of printing to the standard output).

The web service should accept a parameter n>0. For the top 10 airports, n is 10. For the X top airports, n is X 


In [1]:
%%time
import pandas as pd
from datetime import datetime

bookings_file = 'bookings.csv.bz2'
data = pd.read_csv(bookings_file, sep='^', chunksize=1e6)

partial_results = []

for chunk_num, chunk in enumerate(data):
    print('Start processing chunk %d at %s' % (chunk_num, datetime.now()))
    
    chunk = chunk[['arr_port', 'pax', 'year']].dropna()
    chunk = chunk[chunk['year'] == 2013]
    result_from_this_chunk = chunk.groupby('arr_port')['pax'].sum()
    
    partial_results.append(result_from_this_chunk)
    
all_results = pd.concat(partial_results)
total_arrivals = all_results.groupby('arr_port').sum()
sorted_by_arrivals = total_arrivals.sort_values(ascending=False)
sorted_by_arrivals.index = sorted_by_arrivals.index.str.strip()

Start processing chunk 0 at 2018-12-09 19:10:46.240742
Start processing chunk 1 at 2018-12-09 19:11:36.842912
Start processing chunk 2 at 2018-12-09 19:12:27.512042
Start processing chunk 3 at 2018-12-09 19:13:19.117359
Start processing chunk 4 at 2018-12-09 19:14:08.441421


<string>:2: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.


Start processing chunk 5 at 2018-12-09 19:14:59.107336
Start processing chunk 6 at 2018-12-09 19:15:49.125726
Start processing chunk 7 at 2018-12-09 19:16:38.605477
Start processing chunk 8 at 2018-12-09 19:17:31.516156
Start processing chunk 9 at 2018-12-09 19:18:17.877368
Start processing chunk 10 at 2018-12-09 19:18:18.874660
CPU times: user 7min 51s, sys: 27.7 s, total: 8min 19s
Wall time: 8min 28s


## Web Service

http://flask.pocoo.org/docs/1.0/quickstart

In [2]:
from flask import Flask, request
from werkzeug.serving import run_simple

app = Flask('topnairports-service')

@app.route('/topnairports')
def topnairports():
    n = int(request.args.get('n', 10))
    return sorted_by_arrivals.head(n).to_json()
    
run_simple('localhost', 5000, app)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Dec/2018 19:20:16] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2018 19:20:16] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2018 19:20:31] "GET /topnairports?n=5 HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2018 19:20:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2018 19:20:37] "GET /topnairports HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2018 19:20:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2018 19:20:52] "GET /topnairports HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2018 19:20:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Dec/2018 19:20:56] "GET /topnairports?n=3 HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2018 19:20:56] "GET /favicon.ico HTTP/1.1" 404 -


To get the top 5:

http://127.0.0.1:5000/topnairports?n=5